In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from numba import *
from timeit import default_timer as timer
import numpy as np

In [ ]:
output_notebook()
max_iterations = 100
half_width = 750
half_height = 500
min_x = -2.0
max_x = 1.0
min_y = -1.0
max_y = 1.0

width = half_width * 2
height = half_height * 2
dx = (max_x - min_x) / width
dy = (max_y - min_y) / height

def show_image(image):
    p = figure(x_range=(min_x, max_x), y_range=(min_y, max_y))
    p.image(image=[image], x=min_x, y=min_y, dw=max_x-min_x, dh=max_y-min_y, palette="Viridis256")
    show(p)
    
def create_image():
    return np.zeros((height, width), dtype=np.uint8)

Create the Mandelbrot set without numba
---------------------------------------------

In [ ]:
def mandelbrot_pixel(x, y):
    c = complex(x, y)
    z = 0.0j
    for i in range(max_iterations):
        z = z * z + c
        if (z.real * z.real + z.imag * z.imag) >= 4:
            return i

    return 255

def mandelbrot():
    image = create_image()
    for x in range(width):
        real = min_x + x * dx
        for y in range(height):
            imag = min_y + y * dy
            color = mandelbrot_pixel(real, imag)
            image[y, x] = color
    return image

In [ ]:
start = timer()
image = mandelbrot()
stop = timer()
print("Execution took: {} s".format(stop-start))

In [ ]:
show_image(image)

Create the Mandelbrot set with numba
-----------------------------------------

In [ ]:
@jit
def mandelbrot_pixel_numba(x, y):
    c = complex(x, y)
    z = 0.0j
    for i in range(max_iterations):
        z = z * z + c
        if (z.real * z.real + z.imag * z.imag) >= 4:
            return i

    return 255

@jit
def mandelbrot_numba():
    image = create_image()
    for x in range(width):
        real = min_x + x * dx
        for y in range(height):
            imag = min_y + y * dy
            color = mandelbrot_pixel_numba(real, imag)
            image[y, x] = color

    return image

In [ ]:
start = timer()
image = mandelbrot_numba()
stop = timer()
print("Execution took: {} s".format(stop-start))

In [ ]:
show_image(image)

Create the Mandelbrot set with numba (eager compilation)
----------------------------------------------------------------

In [ ]:
@jit(uint8(float64,float64), nopython=True)
def mandelbrot_pixel_numba_eager(x, y):
    c = complex(x, y)
    z = 0.0j
    for i in range(max_iterations):
        z = z * z + c
        if (z.real * z.real + z.imag * z.imag) >= 4:
            return i

    return 255

@jit
def mandelbrot_numba_eager():
    image = create_image()
    for x in range(width):
        real = min_x + x * dx
        for y in range(height):
            imag = min_y + y * dy
            color = mandelbrot_pixel_numba_eager(real, imag)
            image[y, x] = color

    return image

In [ ]:
start = timer()
image = mandelbrot_numba_eager()
stop = timer()
print("Execution took: {} s".format(stop-start))

In [ ]:
show_image(image)

Create the Mandelbrot set with numba (vectorize)
------------------------------------------------------

In [ ]:
@vectorize([uint8(float64, float64)])
def mandelbrot_pixel_numba_vectorize(x, y):
    c = complex(x, y)
    z = 0.0j
    for i in range(max_iterations):
        z = z * z + c
        if (z.real * z.real + z.imag * z.imag) >= 4:
            return i

    return 255

def mandelbrot_numba_vectorize():
    X, Y = np.meshgrid(
        np.arange(min_x, max_x, dx),
        np.arange(min_y, max_y, dy),
    )
    return mandelbrot_pixel_numba_vectorize(X, Y).reshape(X.shape)

In [ ]:
start = timer()
image = mandelbrot_numba_vectorize()
stop = timer()
print("Execution took: {} s".format(stop-start))

In [ ]:
show_image(image)

Create the Mandelbrot set with numba (guvectorize)
---------------------------------------------------------

In [ ]:
@guvectorize([(uint64[:], uint64[:], uint8[:,:])], "(n),(m)->(m,n)")
def mandelbrot_pixel_numba_guvectorize(x_indices, y_indices, image):
    for xi in x_indices:
        x = min_x + xi * dx
        for yi in y_indices:
            y = min_y + yi * dy
            
            c = complex(x, y)
            z = 0.0j
            for i in range(max_iterations):
                z = z * z + c
                if (z.real * z.real + z.imag * z.imag) >= 4:
                    image[yi, xi] = i
                    break
            else:
                image[yi, xi] = 255

def mandelbrot_numba_guvectorize():
    image = np.zeros((height, width), dtype=np.uint8)
    x = np.arange(0, width, dtype=np.uint64)
    y = np.arange(0, height, dtype=np.uint64)
    mandelbrot_pixel_numba_guvectorize(x, y, image)
    return image

In [ ]:
start = timer()
image = mandelbrot_numba_guvectorize()
stop = timer()
print("Execution took: {} s".format(stop-start))

In [ ]:
show_image(image)